# Data Pre-Processing

Is your data too messy to be utilized in 02? Look no further! This notebook walks through the data pre-processing methodology for our datasets, particularly BDD100K. We also include some helpful tips to make your data more compatible with these notebooks.


In [1]:
import networkx as nx
import osmnx as ox 
import time
from shapely.geometry import Polygon
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from algorithms import mm_utils

%matplotlib inline
#ox.__version__

/home/gjgress/.local/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


# Importing your Data

If you are seriously testing your algorithm against data, chances are your dataset is huge. Blindly trying to import it into a Pandas (Geo)DataFrame is going to cause some issues, because it will attempt to load it all into memory (which is likely impossible).

In our case, we use Dask to handle this.

(In general, if you are exclusively using Pandas, Modin might be easier, as it is drop-in compatible. But Dask can handle GeoDataFrames (unlike Modin), so we will use that here) 

In [2]:
# Import the Dask libraries we need
import dask.dataframe as dd
import dask_geopandas as dgpd

# We load all the JSON files

# Unfortunately, our JSON files aren't line-delimited, so I cannot increase partition size.
# As a result, parallelizing will be difficult (each partition is a file)
# But once I do the conversion to the properly formatted JSON, I can ensure it is line-delimited
# And so when I load it later for use, I will be able to parallelize

df = dd.read_json('BDD100K/train/*.json',orient = 'index')
#df.partitions[0].compute()

Great, now we loaded our data into the notebook (you will have to repeat this process within other notebooks-- just copy the cell and put it at the start). But there's still a lot that needs to be done before we can plug in the data into our algorithms. For us, we need to reformat our data into a standard data format.

In [ ]:
gdf = dgpd.from_geopandas(pd.DataFrame([]),npartitions=1)

## TODO: Parallelize the following loop
## Also, maybe don't write to JSON? Whatever format works best for SQLite database

for part in df.partitions:
    row = part.compute().loc['gps'][0]
    temp = gpd.GeoDataFrame({"type": "Feature",
  "geometry": {
    "type": "Point",
    "coordinates": [row["longitude"], row["latitude"]]
  },
  "properties": {
    "timestamp": row["timestamp"],
    "altitude": row["altitude"],
    "speed": row["speed"],
    "vertical accuracy": row["vertical accuracy"],
    "horizontal accuracy": row["horizontal accuracy"]
  }})
    gdf.append(temp)

gdict = {"type": "FeatureCollection", "features": gdf}

In [ ]:
# Now gdict is in GeoJSON format, so we will make it a JSON and export this to SQL database 

gjson = json.dumps(gdict)

#For my testing only
gdf = dd.read_json(gjson,orient = 'records', lines=True, blocksize=1000000) # If I did this right, I now have a Dask gdf with partition size of 1MB

#with open('data.json', 'w') as f:
#  f.write(geojsonfp)

# Enable GeoJSON driver
#fiona.drvsupport.supported_drivers["GeoJSON"] = "r"

#tripdata1 = gpd.read_file("data.json", enabled_drivers="GeoJSON")

In [41]:
geodf.append(gpd.GeoDataFrame(temp))

/home/gjgress/.local/lib/python3.10/site-packages/dask/dataframe/core.py:3088: FutureWarning: The frame.append method is deprecated and will be removed fromdask in a future version. Use dask.dataframe.concat instead.
  warnings.warn(


,type,geometry,properties
npartitions=2,,,
,object,object,float64
,...,...,...
,...,...,...


That looks better. Let's export this to a database so we don't have to repeat this process every time.

In [ ]:
# In our case, it makes more sense to store it into a SQLite database
# Then when we are ready to use it, we can load it smartly and call individual GeoDataFrames from the database


Much better. But there's still other things to check. For example: is your data fused?

In [ ]:
## Display data and see if fused
import sqlite3
from sqlalchemy import create_engine

BDD100K_train = create_engine('sqlite:///BDD100K_train.db')


In our case, our data is already fused. But often you will have several datasets with asynchronous data that you will have to fuse first. We implemented a barebones method in mm_utils to handle this; here is an example of how to apply it.

Note that your data needs to be a (Geo)DataFrame. Also, the first column of all the datasets needs to be the time, and must all share the same time formatting. If you aren't sure your time format will work, we recommmend converting it all to Unix time (most languages have a built-in method to do this)